In [4]:
from PyPDF2 import PdfReader
import os
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora
import gensim
import pandas as pd
import matplotlib.pyplot as plt

from PyPDF2 import PdfReader
import os
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora
import gensim
import pandas as pd
import matplotlib.pyplot as plt

def extract_text_from_pdfs(pdf_directory):
    pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]
    pdf_texts = []
    for pdf_file in pdf_files:
        pdf_reader = PdfReader(open(os.path.join(pdf_directory, pdf_file), 'rb'))
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        pdf_texts.append((pdf_file[:4], text))  # Assuming the first four characters of the file name represent the year
    return pdf_texts

def preprocess_data(doc_set):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        stop_free = ' '.join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized
    return [clean(doc).split() for _, doc in doc_set]

def build_lda_model(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=50)
    return lda_model, corpus, dictionary

def get_dominant_topic(lda_model, corpus, doc_set):
    def format_topics_sentences(ldamodel, corpus):
        sent_topics_df = pd.DataFrame()
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  
                    topic_df = pd.DataFrame([int(topic_num), round(prop_topic,4)], index=['Dominant_Topic', 'Perc_Contribution']).transpose()
                    sent_topics_df = pd.concat([sent_topics_df, topic_df], ignore_index=True)
                else:
                    break
        return(sent_topics_df)
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus)
    # extract the dominant topic for each document
    dominant_topic = df_topic_sents_keywords[['Dominant_Topic', 'Perc_Contribution']]
    # add the year and the original text to the dataframe
    dominant_topic['Year'] = [year for year, text in doc_set]
    dominant_topic['Text'] = [text for year, text in doc_set]
    return dominant_topic

pdf_directory = 'theses_raw'  # The directory that contains your PDFs
doc_set = extract_text_from_pdfs(pdf_directory)
text_data = preprocess_data(doc_set)
lda_model, corpus, dictionary = build_lda_model(text_data)
dominant_topic = get_dominant_topic(lda_model, corpus, doc_set)

df = dominant_topic

df_grouped = df.groupby(['Year', 'Dominant_Topic']).size().unstack(fill_value=0)
df_grouped.plot(kind='bar', stacked=True)
plt.show()



AttributeError: 'DataFrame' object has no attribute 'append'